In [ ]:
#######################################
## SIS 420                           ##
## Alumno: Sanchez Calvimontes Pablo ##
## Carrera: Ingenieria de Sistemas   ##
#######################################

# se uso el dataset de:
# https://www.kaggle.com/datasets/andrewmvd/fetal-health-classification

# este data set es usado para clasificar la salud fetal para prevenir la mortalidad infantil y materna.

# el problema con este data set esque tiene pocos datos asi que primero se creo un dataset sintetico
# para lograr una mejor prediccion del modelo

# el data set tiene 22 columnas
# baseline value
# accelerations
# fetal_movement
# uterine_contractions
# light_decelerations
# severe_decelerations
# prolongued_decelerations
# abnormal_short_term_variability
# mean_value_of_short_term_variability
# percentage_of_time_with_abnormal_long_term_variability
# mean_value_of_long_term_variability
# histogram_width
# histogram_min
# histogram_max
# histogram_number_of_peaks
# histogram_number_of_zeroes
# histogram_mode
# histogram_mean
# histogram_median
# histogram_variance
# histogram_tendency
#-------------
# LA COLUMNA Y (22) ES  :fetal_health


# se realizaron muchas pruebas para obtener una funcion de perdida lo mas minima posible
# los resultados se encuntran abajo
# pero el mejkor resultado tommo 47 minutos aproximandamente

#########################
# con 500 neuronas      #
# 100 epoch             #
# 3 capas ocultas       #
#########################

# se uso 22000 datos para el entrenamiento y el restante se uso para el testeo del modelo

# Perceptrón Multicapa para Clasificacion

## MLP y Capas

Vamos a empezar definiendo nuestra clase `MLP`, esta clase estará formada por una lista de capas

In [ ]:
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando
        # cada capa de manera secuencial
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class Layer():
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolver los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizer
        return

Ahora podemos definir las diferentes capas que utilizaremos. Hasta ahora sólo hemos visto la capa lineal y diferentes funciones de activación.

In [ ]:
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T)
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

In [ ]:
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

## Optimizador

In [ ]:
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

## Funciones de pérdida

In [ ]:
class Loss():
    def __init__(self, net):
        self.net = net

    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)


class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target.astype(int)]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target.astype(int)] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

## Implementación

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/EXAMEN_FINAL_SanchezPablo/ej2/Datasets/20k_datosSinteticos.csv', delimiter=',', skiprows=1)
data = data.to_numpy()

In [ ]:
X = data[:22000 ,:21]
Y = data[:22000, 21]

# normalización datos
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std

X.shape, Y.shape


((22000, 21), (22000,))

### Clasificación Multiclase

In [ ]:
D_in, H, D_out = 21, 500, 4
mlp = MLP([
    Linear(D_in, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, H),
    ReLU(),
    Linear(H, D_out)
])

optimizer = SGD(mlp, lr=0.02)
loss = CrossEntropy(mlp)

epochs = 100
batch_size = 10

batches = len(X) // batch_size
log_each = 10
l = []
for e in range(1,epochs+1):
    _l = []
    for b in range(batches):
        x = X_norm[b*batch_size:(b+1)*batch_size]
        y = Y[b*batch_size:(b+1)*batch_size]
        y_pred = mlp(x)
        _l.append(loss(y_pred, y))
        loss.backward()
        optimizer.update()
    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')

Epoch 10/100, Loss: 0.6245
Epoch 20/100, Loss: 0.5861
Epoch 30/100, Loss: 0.5104
Epoch 40/100, Loss: 0.4285
Epoch 50/100, Loss: 0.3617
Epoch 60/100, Loss: 0.3107
Epoch 70/100, Loss: 0.2728
Epoch 80/100, Loss: 0.2417
Epoch 90/100, Loss: 0.2165
Epoch 100/100, Loss: 0.1955


In [ ]:
#array de prueba
X_test = X[20000:, :].copy()

#Normalizar datos de prueba si es necesario
X_test_norm = (X_test - X_mean) / X_std

# Pasa datos de prueba a trabes del modelo para obtener las predicciones
y_pred_test = mlp(X_test_norm)

#convierte las predicciones a clases
prediccion = np.argmax(y_pred_test, axis=1)

# puedes imprimir las predicciones para verificar resultados
print("Predicciones del modelo: ")
print(prediccion)
print("Valores Realess: ")
print(Y[20000:])

#calcula el porcentaje de relacion entre las predicciones y los valores reales
total_ejemplos = len(prediccion)
correlacion = np.sum(prediccion == Y[20000: ])
porcentaje = (correlacion / total_ejemplos) * 100

#imprime el porcentaje de relaccion
print("Porcentaje de relacion: ")
print(f"{porcentaje:.2f}%")

Predicciones del modelo: 
[1 2 1 ... 1 3 1]
Valores Realess: 
[1. 2. 1. ... 1. 3. 1.]
Porcentaje de relacion: 
99.95%
